### Student Information
Name: Orison

Student ID: 106065425

Dataset Used: Sentiment Labelled Sentesces Data Set (URL:https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences)

### Instructions

- Download the dataset provided in this [link](https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences#). The sentiment dataset contains a `sentence` and `score` label. Read what the dataset is about on the link provided before you start exploring it. 


- Then, you are asked to apply each of the data exploration and data operation techniques learned in the [first lab session](https://goo.gl/Sg4FS1) on the new dataset. You don't need to explain all the procedures as we did in the notebook, but you are expected to provide some **minimal comments** explaining your code. You are also expected to use the same libraries used in the first lab session. You are allowed to use and modify the `helper` functions we provided in the first lab session or create your own. Also, be aware that the helper functions may need modification as you are dealing with a completely different dataset. This part is worth 80% of your grade!


- After you have completed the operations, you should attempt the **bonus exercises** provided in the [notebook](https://goo.gl/Sg4FS1) we used for the first lab session. There are six (6) additional exercises; attempt them all, as it is part of your grade (10%). 


- You are also expected to tidy up your notebook and attempt new data operations that you have learned so far in the Data Mining course. Surprise us! This segment is worth 10% of your grade.


- After completing all the above tasks, you are free to remove this header block and submit your assignment following the guide provided in the `README.md` file of the assignment's [repository](https://github.com/omarsar/data_mining_hw_1). 

# Dataset Information

Sentiments Labelled

This dataset was created for the paper 'From Group to Individual Labels using Deep Features'

It uses a sentence score, either 1 or 0 to determine if the sentence is positive or negative, respectively. They were selected from three different websites. No neutral sentences were intended for selection.



# 0. Importing Libraries

In [1]:
# necessary for when working with external scripts
%load_ext autoreload
%autoreload 2

In [18]:
import pandas as pd
import numpy as np
import nltk


from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
import plotly.plotly as py
import plotly.graph_objs as go
import math
%matplotlib inline

# my functions
import helpers.data_mining_helpers as dmh
import helpers.text_analysis as ta

# 1. Preparing the Data

First let's load some data and see if it works well.

In [57]:
data = pd.read_csv("imdb_labelled.txt", header=0, delimiter= "\t", quoting=3)


In [63]:
print (data)

    A very, very, very slow-moving, aimless movie about a distressed, drifting young man.    \
0    Not sure who was more lost - the flat characte...                                        
1    Attempting artiness with black & white and cle...                                        
2         Very little music or anything to speak of.                                          
3    The best scene in the movie was when Gerardo i...                                        
4    The rest of the movie lacks art, charm, meanin...                                        
5                                  Wasted two hours.                                          
6    Saw the movie today and thought it was a good ...                                        
7                                 A bit predictable.                                          
8    Loved the casting of Jimmy Buffet as the scien...                                        
9                 And those baby owls were adorabl